<a href="https://colab.research.google.com/github/nunsongi/quito-risk-mapping-python/blob/main/lluvia_estaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpieza y estandarización de datasets de lluvia (Quito)

## Objetivo
Unificar y depurar las series de precipitación **horaria, diaria y mensual** de estaciones dentro del **DMQ (Quito)** para el periodo **2019 a 2025**, aplicando controles de calidad y completitud, y exportar **datasets limpios y consistentes** (diario y mensual) listos para análisis de inundaciones y para su integración con el notebook principal.

---

## Contenido
1. Importación de Librerías
2. Carga de datos
